# Finding Miller index of plane

In [41]:
from pymatgen.io.lammps.data import LammpsData
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd 
import transformation as tfm
from functools import reduce

## Get original structure lattice vectors

In [22]:
filename = '/home/kimia.gh/blue2/B4C_ML_Potential/analysis_scripts/data_files/all_atoms/data.B12-CBC'
struct_orig = LammpsData.from_file(filename,atom_style='atomic').structure
coord_axes_orig = [[1,0,0],[0,1,0],[0,0,1]]
A1 = struct_orig.lattice.matrix
print(f'Original Lattice Vectors: \na: {A1[0]}\nb: {A1[1]}\nc: {A1[2]}')

Original Lattice Vectors: 
a: [5.193 0.    0.   ]
b: [2.116065 4.742311 0.      ]
c: [2.116066 1.372958 4.539217]


## Rotate the structure for desired angle
Then get lattice vectors and new coordinate axes vectors

In [23]:
struct_rot,new_coord_axes = tfm.rotate_lattice(struct_orig,0)
A2 = struct_rot.lattice.matrix
print(f'Rotated Lattice Vectors: \na: {A2[0]}\nb: {A2[1]}\nc: {A2[2]}')

Rotated Lattice Vectors: 
a: [ 2.90745421 -1.48292128  4.03917114]
b: [-0.16948041  3.25938996  4.03917048]
c: [-2.7379738  -1.77646869  4.03917052]


## Get transormation matrix from rotated coordinate system to original coordinate system

In [32]:
#Get transformation matrix for rotating cartesian coordinates to align with new_coor_axes
T = np.dot(coord_axes_orig, np.linalg.inv(new_coord_axes))

## Transform plane from new coordinate system back to original coordinate system

$z = -0.01403x - 1.075y+316.8$

In [40]:
#locations where the plane intersects with the rotated x,y,z axes
plane_cart_rot = np.array([316.8/0.01403, 316.8/1.075, 316.8])

#find transformation from coord axes to crystal axes
T_coord_to_crystal = np.dot(A2,np.linalg.inv(new_coord_axes))

#find intersection of plane with crystal axes
plane_crystal = np.dot(T_coord_to_crystal,plane_cart_rot)

#take reciprocals of intersepts
hkl = np.reciprocal(plane_crystal)

hkl


array([8.52813714e-06, 2.03340114e-05, 2.01516233e-05])

In [43]:
import numpy as np
from fractions import Fraction
from math import gcd
from functools import reduce

# Function to find the least common denominator (LCD) of a list of numbers
def lcm(a, b):
    return abs(a * b) // gcd(a, b)

def find_lcd(denominators):
    return reduce(lcm, denominators)

# Function to clear fractions
def clear_fractions(reciprocals):
    # Convert each number to a fraction
    fractions = [Fraction(value).limit_denominator() for value in reciprocals]
    
    # Extract denominators
    denominators = [frac.denominator for frac in fractions]
    
    # Find the least common denominator (LCD)
    lcd = find_lcd(denominators)
    
    # Multiply each fraction by the LCD to get whole numbers
    cleared_fractions = [frac * lcd for frac in fractions]
    
    # Return the whole numbers (numerators)
    return [int(frac.numerator) for frac in cleared_fractions], lcd

# Example usage
reciprocals = hkl  # Example: (1, 1/2, 0) corresponds to intercepts (1, 2, ∞)

# Clear fractions and return whole numbers
whole_numbers, lcd = clear_fractions(reciprocals)

# Output
print("Whole numbers (Miller indices):", whole_numbers)
print("Least common denominator:", lcd)


Whole numbers (Miller indices): [5935132654944, 14151393895872, 14024462570791]
Least common denominator: 695946978094496628


In [52]:
miller = (np.array(whole_numbers).round(2)/5935132654944).round(1)

In [53]:
T_crys_to_cart = np.linalg.inv(T_coord_to_crystal)

In [55]:
plane = np.dot(T_crys_to_cart,np.reciprocal(miller))
plane

array([0.19256692, 0.0019363 , 0.00143723])

In [58]:
(plane/np.linalg.norm(plane)).round(2)

array([1.  , 0.01, 0.01])